# Collaborative Filtering with Matrix Factorization
---  

## Concept
(Non-negative) Matrix Factorization (or Approximation) is a model based collaborative filtering technique
non-negative ➜ because all elements of the matrix are generally positive or zero
matrix factorization ➜ because a large (and sparse) matrix containing all user and rating information is divided into two much smaller matrices/factors

In non-negative matrix factorization, we are trying to factorise (separate) the rating matrix into two matrices, for users and for films separately, each of which also has latent features in the hidden axis. The sub-matrices are found so that their product approximates ratings matrix R.

<img src="nmf.png" width="600" height="400">

## Your task
Complete the NMF worksheet
[Course Materials](https://spiced.space/gradient-masala/ds-course/chapters/project_movie_recommender/model_based_cf.html)

## Coding

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from sklearn.decomposition import NMF
import pickle

#### Let's collect some recommendations for a new users that loves Disney Movies! 

In [ ]:
# for calculating recommendations
query = {
    # movieId, rating
    4470:5, 
    48:5,
    594:5,
    27619:5,
    152081:5,
    595:5,
    616:5,
    1029:5
}


# for testing the recommender after getting some recommendations
relevant_items = [
    596, 4016, 1033, 134853, 
    2018, 588, 364, 26999, 75395, 2085, 
    1907, 2078, 1032, 177765   
]

## Non Negative Matrix Factorization for Recommender Systems
---



In [ ]:
ratings = pd.read_csv('../data/ml-latest-small/ratings.csv')
movies = pd.read_csv('../data/ml-latest-small/movies.csv')

In [ ]:
# which movies are in the query?
movies.set_index('movieId').loc[query.keys()]

---
## 1. Model Development

### Preprocessing

- filter out movies rated by less than 20/ 50 / 100 ... users
- filter out movies with an average rating lower than 2
- create a sparse user item matrix

In [ ]:
# Check original data
ratings

In [ ]:
# calculate the number of ratings per movie
rating_per_movie=ratings.groupby('movieId')['userId'].count()
rating_per_movie

In [ ]:
# filter for movies with more than 20 ratings and extract the index
popular_movie=rating_per_movie.loc[rating_per_movie>20]
popular_movie

In [ ]:
# filter the ratings matrix and only keep the popular movies
ratings=ratings.set_index('movieId').loc[popular_movie.index]
ratings=ratings.reset_index()
ratings

In [ ]:
# Initialize a sparse user-item rating matrix 
# (data, (row_ind, col_ind)
R=csr_matrix((ratings['rating'], (ratings['userId'], ratings['movieId'])))

In [ ]:
R

### Training

- initialize the model
- fit it on the user item matrix
- optionally, tune the number of components (hidden features): what happens if you set the number of components to a really low number?
- decrease the `tol` to train for a longer time

In [ ]:
# initialize the unsupervised model
# 55 hidden features, F=55
model = NMF(n_components=55, init='nndsvd', max_iter=10000, tol=0.01, verbose=2)

# fit it to the user-item rating matrix
model.fit(R)

# initialzed P, Q matrix with random values
# iterate and optimize the values stored in P and Q

### Model inspection

In [ ]:
R

#### the hidden features

In [ ]:
model.components_.shape

In [ ]:
# user-'genre' matrix [611x55]
P =model.transform(R)

# movie-'genre' matrix [55x168253]
Q = model.components_

P.shape, Q.shape

In [ ]:
# user with id 1: sparse format
R[1,:]

In [ ]:
# user with id 1: dense embedding
P[1,:]

In [ ]:
# dense embedding for movie with id 1
Q[:,1]

In [ ]:
# reconstructed matrix Rhat
# R_hat = P.dot(Q)

In [ ]:
# R -> encoding -> P -> decoding -> Rhat
R_hat = model.inverse_transform(model.transform(R))

In [ ]:
R_hat

#### the reconstruction error

$$
L(R, \hat{R}) = \sqrt{\sum_i\sum_j(R_{ij}-\hat{R}_{ij})^2} = \sqrt{\sum_i\sum_j(R_{ij}-PQ_{ij})^2}
$$

In [ ]:
R.shape, R_hat.shape

In [ ]:
# reconstruction error
np.sqrt(np.sum(np.square(R - R_hat)))

In [ ]:
model.reconstruction_err_

---
## 2. Model deployment: Make recommendations for a new user

### Save the trained model on your hard drive

In [ ]:
with open('./nmf_recommender.pkl', 'wb') as file:
    pickle.dump(model, file)

In [ ]:
!ls

### Read the model from hard drive

In [ ]:
with open('./nmf_recommender.pkl', 'rb') as file:
    model = pickle.load(file)

In [ ]:
model.reconstruction_err_

### Receive a user query

In [ ]:
query

In [ ]:
R[1,:]

### Construct a user vector

we need the same input as was used during training!

In [ ]:
list(query.values())

In [ ]:
data=list(query.values())           # the ratings of the new user
row_ind=[0]*len(data)          # we use just a single row 0 for this user
col_ind=list(query.keys())  
data, row_ind,col_ind                           # the columns (=movieId) of the ratings


In [ ]:
# new user vector: needs to have the same format as the training data
user_vec=csr_matrix((data, (row_ind, col_ind)), shape=(1, R.shape[1]))
user_vec


In [ ]:
R

### Calculate the score

1. transform the user vector to its dense representation (encoding) 
2. inverse transform the dense vector into the sparse representation (decoding)

$$
\hat{r}_{ij} = p_i' \cdot q_j 
$$

In [ ]:
# user_vec -> encoding -> p_user_vec -> decoding -> user_vec_hat

scores=model.inverse_transform(model.transform(user_vec))


# convert to a pandas series
scores=pd.Series(scores[0])
scores

### Give recommendations

In [ ]:
query.keys()

In [ ]:
# give a zero score to movies the user has allready seen
scores[query.keys()]=0

In [ ]:
# sort the scores from high to low 
scores=scores.sort_values(ascending=False)
scores

In [ ]:
# get the movieIds of the top 10 entries
recommendations=scores.head(10).index
recommendations

In [ ]:
movies.set_index('movieId').loc[recommendations]

---
## 3. Project Task: NMF recommender function

1. Collect different user queries for "typical" users (e.g. a horror movie buff) and evaluate the algorithm
2. Set the number of components to a very low number (e.g. 2). What happens to the recommendations?
3. Implement a recommender function that recommends movies to a new user based on the NMF model!

Note: Training of the model happens outside of the function! Don't retrain the model every time you want to calculate recommendations for a user.


In [ ]:
# collaborative filtering = look at ratings only!
def recommend_nmf(query, model, ratings, k=10):
    """
    Filters and recommends the top k movies for any given input query based on a trained NMF model. 
    Returns a list of k movie ids.
    """
    # 1. candiate generation
    
    # construct a user vector
    
   
    # 2. scoring
    
    # calculate the score with the NMF model
    
    
    # 3. ranking
    
    # filter out movies allready seen by the user
    
    # return the top-k highst rated movie ids or titles
    
    return [364, 372, 43, 34, 243]

In [ ]:
# recommender.py
# from recommender import recommend_nmf